In [1]:
%run model.ipynb
%run Covid19AntigenDataset.ipynb

Using cpu device
WORKS@!
torch.Size([180])


In [2]:
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix
from torch.utils.data.sampler import WeightedRandomSampler

dataset = Covid19AntigenDataset()

train_size = int(0.9 * dataset.__len__())
test_size = dataset.__len__() - train_size

train_set, test_set = torch.utils.data.random_split(dataset, [train_size, test_size])

learning_rate = 1e-4
train_batch_size = 64
test_batch_size = 64

class_weights = [0.1,1]
train_labels = [dataset.data[i][1] for i in train_set.indices]
class_weights = [class_weights[label] for label in train_labels]
sampler = WeightedRandomSampler(torch.DoubleTensor(class_weights), len(class_weights))

# train_dataloader = DataLoader(train_set, batch_size=train_batch_size,sampler=sampler)
train_dataloader = DataLoader(train_set, batch_size=train_batch_size,shuffle=True)
test_dataloader = DataLoader(test_set, batch_size=test_batch_size, shuffle=True)

# model = NeuralNetwork(216)
model = NeuralNetwork(180)

# weights = torch.Tensor([1,0.9])
# loss_fn = BCELoss_class_weighted(weights)
loss_fn = nn.BCELoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

WORKS@!
NEW!@@!


In [3]:
def train_loop(dataloader, model, loss_fn, optimizer):
    loop_loss = 0 
    size = len(dataloader.dataset.indices)
    correct = 0
    pred_y = torch.Tensor([])
    true_y = torch.Tensor([])
        
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        X = X.float()
        y = y.float()

        pred = model(X)
        pred_y = torch.cat((pred_y,(torch.flatten(pred) + 0.5).int()), 0)
        true_y = torch.cat((true_y,torch.flatten(y)), 0)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loop_loss += loss.item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    correct /= size
    num_batches = len(dataloader)
    loop_loss /= num_batches
    print(f"Train Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {loop_loss:>8f} \n")
    return (true_y, pred_y, loop_loss)


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss = 0
    correct = 0
    pred_y = torch.Tensor([])
    true_y = torch.Tensor([])

    with torch.no_grad():
        for X, y in dataloader:
            X = X.float()
            y = y.float()
            pred = model(X)
            
            pred_y = torch.cat((pred_y,(torch.flatten(pred) + 0.5).int()), 0)
            true_y = torch.cat((true_y,torch.flatten(y)), 0)
            
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return (true_y, pred_y, test_loss)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score

epochs = 40
test_losses, test_accuracies, test_f1_scores, test_bas = [], [], [], []
train_losses, train_accuracies, train_f1_scores, train_bas = [], [], [], []

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_true_y, train_pred_y, train_loss = train_loop(train_dataloader, model, loss_fn, optimizer)
    test_true_y, test_pred_y, test_loss = test_loop(test_dataloader, model, loss_fn)
    
    test_accuracy = accuracy_score(test_true_y, test_pred_y, normalize=True)
    train_accuracy = accuracy_score(train_true_y, train_pred_y, normalize=True)
    
    test_f1 = f1_score(test_true_y, test_pred_y)
    train_f1 = f1_score(train_true_y, train_pred_y)
    
    print(torch.unique(test_pred_y))
    
    test_ba = balanced_accuracy_score(test_true_y, test_pred_y)
    train_ba = balanced_accuracy_score(train_true_y, train_pred_y)
    
    test_losses += [test_loss]
    test_accuracies += [test_accuracy]
    test_f1_scores += [test_f1]
    test_bas += [test_ba]
    
    train_losses += [train_loss]
    train_accuracies += [train_accuracy]
    train_f1_scores += [train_f1]
    train_bas += [train_ba]

print("Done!")

Epoch 1
-------------------------------


/Users/dort/opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
/Users/dort/opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([30])) that is different to the input size (torch.Size([30, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
/Users/dort/opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([61])) that is different to the input size (torch.Size([61, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, red

Train Error: 
 Accuracy: 89.0%, Avg loss: 0.298036 

Test Error: 
 Accuracy: 90.1%, Avg loss: 0.188751 

tensor([0., 1.])
Epoch 2
-------------------------------
Train Error: 
 Accuracy: 89.0%, Avg loss: 0.175293 

Test Error: 
 Accuracy: 90.1%, Avg loss: 0.157425 

tensor([0., 1.])
Epoch 3
-------------------------------
Train Error: 
 Accuracy: 89.0%, Avg loss: 0.156463 

Test Error: 
 Accuracy: 90.1%, Avg loss: 0.154336 

tensor([0., 1.])
Epoch 4
-------------------------------
Train Error: 
 Accuracy: 89.0%, Avg loss: 0.149844 

Test Error: 
 Accuracy: 90.1%, Avg loss: 0.153478 

tensor([0., 1.])
Epoch 5
-------------------------------
Train Error: 
 Accuracy: 89.0%, Avg loss: 0.143951 

Test Error: 
 Accuracy: 90.1%, Avg loss: 0.154381 

tensor([0., 1.])
Epoch 6
-------------------------------
Train Error: 
 Accuracy: 89.0%, Avg loss: 0.138650 

Test Error: 
 Accuracy: 90.1%, Avg loss: 0.153195 

tensor([0., 1.])
Epoch 7
-------------------------------
Train Error: 
 Accuracy: 89.

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
x = range(1, epochs +1)

plt.plot(x, test_accuracies, label = "test")
plt.plot(x, train_accuracies, label = "train")
plt.ylabel('Accuracies')
plt.xlabel('epoch')
plt.legend()
plt.show()
fig.savefig('accuracies.png')

plt.plot(x, test_f1_scores, label = "test")
plt.plot(x, train_f1_scores, label = "train")
plt.ylabel('F1 score')
plt.xlabel('epoch')
plt.legend()
plt.show()
fig.savefig('f1_score.png')

plt.plot(x, test_bas, label = "test")
plt.plot(x, train_bas, label = "train")
plt.ylabel('Balanced Accuracies score')
plt.xlabel('epoch')
plt.legend()
plt.show()
fig.savefig('balanced_accuracies.png')

fig = plt.figure()
plt.plot(x, test_losses, label = "test")
plt.plot(x, train_losses, label = "train")
plt.ylabel('Losses')
plt.xlabel('epoch')
plt.legend()
plt.show()
fig.savefig('losses.png')